# AI bot with UI

In [1]:
import random 
import json
import string

import numpy as num

import nltk
from nltk.stem import WordNetLemmatizer 

import tensorflow as tf
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam
#nltk.download("punkt")
#nltk.download("wordnet")

#import pickle

import warnings
warnings.filterwarnings('ignore')

Data: intents JSON file

If you already have an intents.json file, load the file:

file = open('intents.json').read()
data = json.loads(file)


Otherwise, create an intents JSON file that lists all possible outcomes of a user interactions with the chatbot

User's queries:

- tag that categorizes a query
- patterns of a query
- bot responses


In [2]:
data = {"intents": [
        {"tag": "greeting",
         "patterns": ["Hi", "Hello", "Good morning", "Good afternoon"],
         "responses": ["Hello", "Hi", "How can I help?"],
         "context_set": ""
        },
        {"tag": "service",
              "patterns": ["what type of financial service do you provide?", "I need help with ABC ?"],
              "responses": ["We provide a wide range of ABC services", "We offer a wide range of ABC services"]
             },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye"],
         "responses": ["Hope to see you again", "Have a nice day", "Bye"]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That was helpful"],
         "responses": ["We are happy to help you!", "You are welcome!", "Thank you for visiting us"]
        },
        {"tag": "hours",
         "patterns": ["What are the open hours?", "What are the hours?", "When are you open?" ],
         "responses": ["We're open Monday to Friday from 9am to 5pm", "Our business hours are 9am-5pm Monday to Friday"]
        },
        {"tag": "payments",
         "patterns": ["Do you take credit cards?", "Do you accept credit cards?", "Can I pay by credit card?" ],
         "responses": ["We accept credit cards", "Yes, we accept most major credit cards"]
        }, 
]
       }

### Processing data

Use a nested for loop to extract the tokens from patterns into a words list
Then create X, Y lists of patterns and corresponding tags

In [3]:
categories = []
words = [] #tokens from patterns
X = [] #patterns
Y = [] #tags


for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern) # use word_tokenize to tokenize 
        words.extend(tokens)
        X.append(pattern)
        Y.append(intent["tag"])
    
    if intent["tag"] not in categories:
        categories.append(intent["tag"])

In [4]:
categories= sorted(set(categories)) #list
print(categories)

['goodbye', 'greeting', 'hours', 'payments', 'service', 'thanks']


Remove punctuation, then lowercase and Lemmatize the words (ie create base words) from the patterns to simplify the words list

In [5]:
wordnet = WordNetLemmatizer() #use WordNet Lemmatizer to lemmatize

words = [wordnet.lemmatize(w.lower()) for w in words if w not in string.punctuation]
words = sorted(set(words)) #list
print(words)

['abc', 'accept', 'afternoon', 'are', 'by', 'bye', 'can', 'card', 'credit', 'do', 'financial', 'good', 'goodbye', 'hello', 'help', 'helpful', 'hi', 'hour', 'i', 'later', 'morning', 'need', 'of', 'open', 'pay', 'provide', 'see', 'service', 'take', 'thank', 'thanks', 'that', 'the', 'type', 'wa', 'what', 'when', 'with', 'you']


### BoW

Constructing a BoW model based on the word counts: each index position in the feature vectors corresponds to the integer 
value 1, if word match found in current pattern and 0 otherwise


In [6]:
train = [] 
outEmpty = [0] * len(categories)

for idx, d in enumerate(X):
    bag = []
    text = wordnet.lemmatize(d.lower())
    for w in words:
        bag.append(1) if w in text else bag.append(0)
    
    outputRow = list(outEmpty)
    outputRow[categories.index(Y[idx])] = 1
    train.append([bag, outputRow])
    
random.shuffle(train)
train = num.array(train, dtype=object)

### AI model 

In [7]:
# preparing the x,y for machine learning

x = num.array(list(train[:, 0]))
y = num.array(list(train[:, 1]))

In [8]:
input_shape = (len(x[0]),)
output_shape = len(y[0])


model = Sequential()
model.add(Dense(100, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))

sgd = SGD(lr=0.01, decay=1e-7, momentum=.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])


print(model.summary())
model.fit(x, y, epochs=208, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4000      
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 306       
                                                                 
Total params: 9,356
Trainable params: 9,356
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/208
1/1 [==============================] 

Build a AI model that will select an appropriate response to query

In [7]:
input_shape = (len(x[0]),)
output_shape = len(y[0])


model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))

adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

print(model.summary())
model.fit(x, y, epochs=208, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5120      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 13,766
Trainable params: 13,766
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/208
1/1 [==============================

Training the NN using Keras: we initialized a new model using the Sequential class to implement a 
Feed Forward Neural Network, then added layers to it.  The input dimension must match the input_shape 
in the training set. We added 2 hidden layers with 100 and 50 hidden units respectively. The number of output units and input units of 2 consecutive layers must also match. The number of units in the output layer must match the
unique category labels, the output_shape.

The loss function was set to categorical cross-entropy, and used softmax for multi-class predictions. After compiling the model, we trained the model by calling the fit method. 

We chose a SGD (Stochastic Gradient Descent) optimization, and set values for the weight decay constant and momentum learning to adjust the learning rate. We set the cost (or loss) function to categorical_crossentropy. After compiling the model, we trained it by calling the fit method on 208 epochs.

Turning the AI model into an AI chatbot 

In [9]:
def Text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [wordnet.lemmatize(w) for w in tokens]
  return tokens

def Bag(text, V): 
  tokens = Text(text)
  bag = [0] * len(V)
  for t in tokens: 
    for idx, w in enumerate(V):
      if w == t: 
        bag[idx] = 1
  return num.array(bag)


In [10]:
def Predict_category(text, vocab, labels):
  bag = Bag(text, vocab)
  Result = model.predict(num.array([bag]))[0]
  thresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(Result) if res > thresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

In [11]:
def get_response(fL, fJ):
  tag = fL[0]
  listIntents = fJ["intents"]
  for i in listIntents:
    if i["tag"] == tag:
        answer = random.choice(i["responses"])
        break
  return answer

### Testing the chatbot:

In [ ]:
while True:
    query = input("")
    intents = Predict_category(query, words, categories)
    answer = get_response(intents, data)
    print(answer)

Hi
1/1 [==============================] - 0s 156ms/step
How can I help?
open
1/1 [==============================] - 0s 31ms/step
We're open Monday to Friday from 9am to 5pm
credit?
1/1 [==============================] - 0s 16ms/step
We accept credit cards


How it works: When a user enter a query, the text data is cleaned, then using a BoW model, the cleaned text is converted into numerical values, which then using the AI model, the bot will predict a tag from the intent file that most closely represents the query, based on which the bot will return a answer from the list of responses.

We could create a GUI for the bot in Python using tkinter, 

GUI programming in Python

Tkinter documentation
https://docs.python.org/3/library/tk.html
